In [1]:
import os

In [2]:
%pwd

'/Users/paramatephuengtrakul/project2023/MLengineer/Chicken-disease-classification/research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'/Users/paramatephuengtrakul/project2023/MLengineer/Chicken-disease-classification'

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [6]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories

In [7]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root]) # self.config return configBox object. So artifact is a key

    

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir
        )

        return data_ingestion_config


In [8]:
import os
import urllib.request as request
import zipfile
from cnnClassifier import logger
from cnnClassifier.utils.common import get_size

In [9]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")

    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, "r") as zip_ref:
            zip_ref.extractall(unzip_path)

In [10]:
# pipeline
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    e

[2023-08-04 17:23:33,295: INFO: common: yaml file: config/config.yaml loaded successfully]
[2023-08-04 17:23:33,298: INFO: common: yaml file: params.yaml loaded successfully]
[2023-08-04 17:23:33,299: INFO: common: create directory at: artifacts]
[2023-08-04 17:23:33,301: INFO: common: create directory at: artifacts/data_ingestion]
[2023-08-04 17:23:36,702: INFO: 3110301831: artifacts/data_ingestion/data.zip download! with following info: 
Connection: close
Content-Length: 14247678
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "a8df77b3d922a7cc1252ca6e17e73cb41dc22b1b6a2f5915c2fcfd52c1f30cf0"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: FB64:0288:DC5AB:FA655:64CCD1A6
Accept-Ranges: bytes
Date: Fri, 04 Aug 2023 10:23:35 GMT
Via: 1.1 varnish
X-Served-By: cache-bkk2310030-BKK
X-Cache: MISS